In [169]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [170]:
#https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports

url = "https://www.ons.gov.uk/file?uri=%2feconomy%2fnationalaccounts%2fbalanceofpayments%2fdatasets%2fuktradecountrybycommodityexports%2fcurrent/countrybycommodityexports1.zip"

In [171]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

In [ ]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Export")

In [ ]:
df.head()

In [ ]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "exports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.exports = df.exports*1000000

In [ ]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"exports": "sum"})

In [ ]:
topcntry = test.sort_values(by = "exports", ascending = False).iloc[0:23].index.to_list()

In [ ]:
df = df[df.COUNTRY.isin(topcntry)]

In [ ]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [ ]:
df

In [ ]:
grp = df.groupby(["COUNTRY"])

In [ ]:
def ex_metal_total(df):
        
    if not (df[df["COMMODITY"] == "9 Unspecified goods"].empty):
                
        foo = pd.DataFrame([])

        fooall = df[df["COMMODITY"] == "Total"].set_index(["time"])["exports"] 
        
        foobar = df[df["COMMODITY"] == "9 Unspecified goods"].set_index(["time"])["exports"]

        foo["exports"] = fooall - foobar
    # Grab the total and then sbutract off the unspecified goods
    
        foo["COUNTRY"] = df.COUNTRY.unique()[0]
    
        #print(df.COUNTRY.unique())
    
        foo["COMMODITY"] = "Total (ex. metals)"
        
        foo.reset_index(inplace = True)
        
        foo[df.columns]
        
        df = df.append(foo)
    
    return df

In [ ]:
df = grp.apply(ex_metal_total)

In [ ]:
df.drop(["COUNTRY"], axis = 1, inplace = True)

df.reset_index(inplace = True)

df.drop(["level_1"], axis = 1, inplace = True)

In [ ]:
file = ".\\data"+ "\\UK-exports-1997-2017.parquet"

foo = pq.read_table(file).to_pandas()

In [ ]:
df = df.append(foo) #add the historical data

In [ ]:
df

In [ ]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [ ]:
df.sort_index(inplace = True)

df.tail()

In [ ]:
df.head()

In [ ]:
out_file = ".\\data"+ "\\UK-exports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [ ]:
file = ".\\data"+ "\\UK-exports.parquet"

foo = pq.read_table(file).to_pandas()

In [ ]:
foo.head()